# Notes
* 获取senti words
* 别忘了clean里面的符号（用clean代替一些代码）
* 拿了大概五百个
* filter weights用sent2net里的词汇
* 从sentiwordnet,如何从文件里抓取极性词，分数大于0.8，建立两个得到list的情感词。分开positive和negative
* 加到一个list
* layer = return true

* 自己得到cnn_filter weights
* 默认train的时候得到weights是initilizer randomly的
* 自己train的时候，trainable为false，因为已经得到情感词
* weight.shape（window_size,feature_dimension(embedding),number of words/number of filter）
* bias.shape(number of sentiment words/number of filter)一维list 
* 两个合在一起

## CNN
* function:get the sentiment words,use the filters to get
* input:()
* filter:to extract some features
* filter_length:the quantity of filters
* max_pooling:
* output:(batch_size*)
* 用两个CNN，一个是knowledge的，如果没找完，就要用random了

## LSTM
* function:
* input:
* output:
* 

# questions
* 为什么maxpooling得到这个
* 可以试试别的
* filter的作用

#### Test
* 两个最重要的是vocab2int 和 embedding matrix
* 纯CNN跑，用全部的数据，不用分句
* 后面三个是以句子为单位
* cnn layer2是基于知识的
* 用skLearn得到recall 和precision


In [2]:
import pickle
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.embeddings import Embedding
from keras.layers import Convolution1D, MaxPooling1D, GlobalMaxPooling1D, Input, Dense, Reshape, LSTM
from keras.layers.merge import concatenate, add
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
import tensorflow as tf
import re
import gensim
import numpy as np
from util.util_functions import getWordIdx

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#loading the train_copus_padded data from .pickle file
file = open('pickle_data/train_copus_pad.pickle','rb')
train_copus_padded = pickle.load(file)

file = open('pickle_data/test_copus_pad.pickle','rb')
test_copus_padded = pickle.load(file)

file = open('pickle_data/vocab_train.pickle','rb')
vocab_to_int_train = pickle.load(file)

file = open('pickle_data/embedding_matrix','rb')
embedding_matrix = pickle.load(file)

file = open('pickle_data/train_label.pickle','rb')
train_label = pickle.load(file)

file = open('pickle_data/test_label.pickle','rb')
test_label = pickle.load(file)
file.close()

In [18]:
print('train test data shape:',train_copus_padded.shape, test_copus_padded.shape)
print('embedding_matrix shape:', embedding_matrix.shape)
#the size of vocabulary
vocab_size = len(vocab_to_int_train)
print('vocabulary size:', vocab_size)
# the maximal length of every sentence
maxlen_sent = train_copus_padded.shape[1]
maxlen_word = train_copus_padded.shape[2]
print('max sent length:', maxlen_sent, '\nmax word length:', maxlen_word)

train test data shape: (25000, 36, 224) (25000, 36, 224)
embedding_matrix shape: (97162, 300)
vocabulary size: 97162
max sent length: 36 
max word length: 224


## Extracting sentiment words from SentiWordNet

In [4]:
file_path = 'resource/SentiWordNet_3.0.0_20130122.txt'
content = []
with open(file_path,'r',encoding='UTF-8') as f:
    while(1):
        lines = f.readline()
        if(lines[0] is ('a' or 'n' or 'v' or 'r')):
            content.append(lines.split('\t'))
        elif(lines[0] is '#'):
            continue
        else:
            break
             
#get the sentiment words(score>0.8)
neg_words1 = []
pos_words1 = []
for lines in content:
    if(float(lines[2])>=0.8):
        pos_words1.append(lines[4])
    elif(float(lines[3])>=0.8):
        neg_words1.append(lines[4])

#get the positive sentiment words
#seperate the multiple words
pos_words2 = []
for i in range(len(pos_words1)):
    pos_words2.append(pos_words1[i].split(' '))

#get the one dimension elements
pos_words3 = []
for i in pos_words2:
    for j in i:
        pos_words3.append(j)

#delete the '#4'
pos_words4 = []
for elem in pos_words3:
    pos_words4.append(re.sub('#\d','',elem))
    
#delete the 'number'
pos_lexicon = []
for elem in pos_words4:
    pos_lexicon.append(re.sub('\d','',elem))

    
#get the negative sentiment words
#seperate the multiple words
neg_words2 = []
for i in range(len(neg_words1)):
    neg_words2.append(neg_words1[i].split(' '))

#get the one dimension elements
neg_words3 = []
for i in neg_words2:
    for j in i:
        neg_words3.append(j)

#delete the '#4'
neg_words4 = []
for elem in neg_words3:
    neg_words4.append(re.sub('#\d','',elem))
    
#delete the 'number'
neg_lexicon = []
for elem in neg_words4:
    neg_lexicon.append(re.sub('\d','',elem))

#merge the positive and negative words
senti_lexicon = pos_lexicon + neg_lexicon
#shuffule the these lists
from sklearn.utils import shuffle 
senti_lexicon = shuffle(senti_lexicon,random_state = 0)

In [5]:
senti_lexicon[:100]

['nifty',
 'swingeing',
 'low',
 'spastic',
 'ill-famed',
 'negative',
 'depressed',
 'banner',
 'condemnable',
 'rosy-cheeked',
 'spasmodic',
 'unforbearing',
 'perverted',
 'kind',
 'scrimy',
 'plain',
 'woebegone',
 'lamentable',
 'idealized',
 'substitute',
 'splendid',
 'unlikable',
 'awe-inspiring',
 'wretched',
 'henpecked',
 'denigrating',
 'cheesy',
 'aesthetic',
 'mild',
 'ruthless',
 'tall',
 'faux',
 'inerrable',
 'thoroughgoing',
 'imitation',
 'fake',
 'burned',
 'atrocious',
 'censurable',
 'dominated',
 'deplorable',
 'dewy-eyed',
 'fearful',
 'gross',
 'preternatural',
 'inapposite',
 'pretty',
 'rugged',
 'pitiful',
 'firm',
 'infamous',
 'neat',
 'mucky',
 'sheltered',
 'poor',
 'inspired',
 'uncivil',
 'unhealthy',
 'scurrilous',
 'clean-living',
 'sorry',
 'musty',
 'unfriendly',
 'greasy',
 'deplorable',
 'healing',
 'diabolic',
 'burned-out',
 'malodourous',
 'righteous',
 'vixenish',
 'charming',
 'miserable',
 'ill-smelling',
 'unthreatening',
 'wretched',
 'in

In [14]:
#map the sentiment words to integer based on vocab2int
senti2int = [getWordIdx(word, vocab_to_int_train) for word in senti_lexicon]

#get the filter weights based on the sentiment words&vocab2int&embedding_matrix
def Find_Filter_Weight(senti2int):
    """sentiwords is the list"""
    word_filter_weights = []
    bias_weights = []
    filter_len = 1
    for i in senti2int:
        vector = embedding_matrix[i]  # shape: 300
        vector = np.expand_dims(vector, axis=0) #shape: 1x 300
        vector = np.expand_dims(vector, axis=2) #shape: 1x 300 x 1
        if len(word_filter_weights) == 0:
            word_filter_weights = vector
        else:
            word_filter_weights = np.concatenate((word_filter_weights, vector), axis=2)   
    #shape is (1, 300, 533)
    
    bias_weights = np.zeros(len(senti2int))
    cnn_wordfilter_weights = [word_filter_weights, bias_weights]
    
    return cnn_wordfilter_weights    

In [16]:
CNN_weights = Find_Filter_Weight(senti2int)
CNN_weights[0].shape

(1, 300, 533)


(1, 300, 533)

## Keras Model

In [19]:
doc_input_padded = Input(shape=(maxlen_sent,maxlen_word), dtype='int32', name='doc_input_padded')
#每个review的维度
embedding_layer = Embedding(vocab_size, embedding_matrix.shape[1], input_length=maxlen_word, 
                            weights=[embedding_matrix], trainable=False)
#embedding matrix shape[1]是300，每个vector的维度
max_pooling_layer = GlobalMaxPooling1D()

def slice(x, index):
    """ Define a tensor slice function
    """
    return x[:, index, :]

#应该使用自定义的层
#要不然就每次都搭建一个层
cnn_layer1 = Convolution1D(nb_filter=100,
                            filter_length=3,
                            border_mode='same',
                            activation='tanh',
                            subsample_length=1)

cnn_layer2 = Convolution1D(nb_filter=CNN_weights[0].shape[2],
                            filter_length=1,
                            border_mode='same',
                            activation='tanh',
                           weights = CNN_weights,
                           trainable = False,
                            subsample_length=1)
#************
stack_layer = Lambda(lambda x: K.stack(x, axis=1))

# interate through sentences in a document
cnn_out = []
for i in range(maxlen_sent):
    #以每个影评的每个句子为输入
    sent = Lambda(slice, arguments={'index': i,})(doc_input_padded)
    sent_embedding = embedding_layer(sent)#input shape:(padded_sentence_number),output shape:(nb_words_padded,dimension)
    
    sent_cnn1 = cnn_layer1(sent_embedding) # output shape: (None, maxlen_word, nb_filter)
    # we use standard max over time pooling
    sent_cnn1 = max_pooling_layer(sent_cnn1)  # output shape: (None, nb_filter)
    
    sent_cnn2 = cnn_layer2(sent_embedding) # output shape: (None, maxlen_word, nb_filter)
    # we use standard max over time pooling
    sent_cnn2 = max_pooling_layer(sent_cnn2)  # output shape: (None, nb_filter)
    
    sent_cnn = concatenate([sent_cnn1, sent_cnn2])
    
    cnn_out.append(sent_cnn)
cnn_out = stack_layer(cnn_out)  # out shape: (None, maxlen_sent, nb_filter)



#处理句子，从头到尾
lstm = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(cnn_out)
dense = Dense(1, activation='sigmoid')(lstm)

model = Model(inputs=[doc_input_padded], outputs=[dense])
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=100, kernel_size=3, strides=1, padding="same")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", weights=[array([[[..., trainable=False, filters=533, kernel_size=1, strides=1, padding="same")`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
doc_input_padded (InputLayer)   (None, 36, 224)      0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 224)          0           doc_input_padded[0][0]           
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 224)          0           doc_input_padded[0][0]           
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 224)          0           doc_input_padded[0][0]           
__________________________________________________________________________________________________
lambda_5 (

In [20]:
batch_size = 100
epoch_num = 10

print('Training...')
model.fit(train_copus_padded, train_label,
          batch_size=batch_size,
          epochs=epoch_num,
          validation_data=(test_copus_padded, test_label))

score, acc = model.evaluate(test_copus_padded, test_label,
                            batch_size=batch_size)
print('Test accuracy:', acc)

Training...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
 5400/25000 [=====>........................] - ETA: 58:47 - loss: 0.6838 - acc: 0.5631

KeyboardInterrupt: 